In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_excel('Maharashtra_Wheat.xls')
df.head()

,District,Year,Season,Area (Hectare),Production (Tonnes),Yield (Tonnes/Hectare)
0,1.AHMEDNAGAR,1997-98,Rabi,79700.0,87100.0,1.092848
1,NaN,1998-99,Rabi,124400.0,154200.0,1.239550
2,NaN,1999-00,Rabi,127000.0,178800.0,1.407874
3,NaN,2000-01,Rabi,94700.0,135800.0,1.434002
4,NaN,2001-02,Rabi,93800.0,156300.0,1.666311


In [3]:
df['District'] = df['District'].fillna(method='ffill')
df.head()

,District,Year,Season,Area (Hectare),Production (Tonnes),Yield (Tonnes/Hectare)
0,1.AHMEDNAGAR,1997-98,Rabi,79700.0,87100.0,1.092848
1,1.AHMEDNAGAR,1998-99,Rabi,124400.0,154200.0,1.239550
2,1.AHMEDNAGAR,1999-00,Rabi,127000.0,178800.0,1.407874
3,1.AHMEDNAGAR,2000-01,Rabi,94700.0,135800.0,1.434002
4,1.AHMEDNAGAR,2001-02,Rabi,93800.0,156300.0,1.666311


In [4]:
import numpy as np

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = np.percentile(df['Area (Hectare)'], 25)
Q3 = np.percentile(df['Area (Hectare)'], 75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define the threshold for identifying outliers
threshold = 1.5

# Calculate the lower and upper bounds for outliers
lower_bound = Q1 - threshold * IQR
upper_bound = Q3 + threshold * IQR

# Identify outliers
outliers = df[(df['Area (Hectare)'] < lower_bound) | (df['Area (Hectare)'] > upper_bound)]

# Remove outliers from the dataset
data_cleaned = df[~((df['Area (Hectare)'] < lower_bound) | (df['Area (Hectare)'] > upper_bound))]

# Print the outliers
print("Outliers:")
print(outliers)

# Print the cleaned dataset
print("Cleaned Dataset:")
print(data_cleaned)


Outliers:
          District      Year       Season  Area (Hectare)  \
1     1.AHMEDNAGAR   1998-99  Rabi               124400.0   
2     1.AHMEDNAGAR   1999-00  Rabi               127000.0   
3     1.AHMEDNAGAR   2000-01  Rabi                94700.0   
4     1.AHMEDNAGAR   2001-02  Rabi                93800.0   
7     1.AHMEDNAGAR   2004-05  Rabi                89300.0   
8     1.AHMEDNAGAR   2005-06  Rabi               103100.0   
9     1.AHMEDNAGAR   2006-07  Rabi               153900.0   
10    1.AHMEDNAGAR   2007-08  Rabi               130900.0   
11    1.AHMEDNAGAR   2008-09  Rabi               138200.0   
12    1.AHMEDNAGAR   2009-10  Rabi               129500.0   
13    1.AHMEDNAGAR   2010-11  Rabi               133700.0   
157     7.BULDHANA   2016-17  Rabi               103880.0   
379      17.NAGPUR   2013-14  Rabi                86200.0   
380      17.NAGPUR   2014-15  Rabi                88500.0   
381      17.NAGPUR   2015-16  Rabi                92600.0   
382      17.NA

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Perform label encoding for 'District' and 'Season' columns
df['District'] = label_encoder.fit_transform(df['District'])
df['Season'] = label_encoder.fit_transform(df['Season'])
df['Year'] = label_encoder.fit_transform(df['Year'])

In [6]:
X = df.drop(['Production (Tonnes)','Yield (Tonnes/Hectare)'],axis=1)
y = df['Production (Tonnes)']

In [7]:
print(df['Year'].unique())
print(df['Year'].isnull().sum())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
0


In [8]:
# df['Year'] = df['Year'].str.strip()

In [9]:
test_years = ['21', '22']

# Split data into training and test sets based on the test years
train_data = df[~df['Year'].isin(test_years)]
test_data = df[df['Year'].isin(test_years)]

# Separate features and target variables
X_train = train_data[['District', 'Year', 'Season', 'Area (Hectare)']]
y_train = train_data['Production (Tonnes)']
X_test = test_data[['District', 'Year', 'Season', 'Area (Hectare)']]
y_test = test_data['Production (Tonnes)']

# Display the shapes of training and test sets
print("Training data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Training data shape: (620, 4) (620,)
Test data shape: (60, 4) (60,)


In [10]:
## Order will matter for knn
X_train = X_train.sort_index()
y_train = y_train.sort_index()
X_test = X_test.sort_index()
y_test = y_test.sort_index()

In [11]:
X_train

,District,Year,Season,Area (Hectare)
0,0,0,0,79700.0
1,0,1,0,124400.0
2,0,2,0,127000.0
3,0,3,0,94700.0
4,0,4,0,93800.0
...,...,...,...,...
673,25,16,0,34500.0
674,25,17,0,29100.0
675,25,18,0,35600.0
676,25,19,0,60200.0


In [12]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform it
X_train = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test = scaler.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [14]:
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
linear_reg_predictions = linear_reg_model.predict(X_test)

In [15]:
linear_reg_mse = mean_squared_error(y_test, linear_reg_predictions)
print("Linear Regression MSE:", linear_reg_mse)

r2_lr= r2_score(y_test, linear_reg_predictions)
print("R-squared score:", r2_lr)

Linear Regression MSE: 436406065.824742
R-squared score: 0.834348459427565


In [16]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)

In [17]:
dt_mse = mean_squared_error(y_test, dt_predictions)
print("Decision Tree Regression MSE:", dt_mse)

r2_dt = r2_score(y_test, dt_predictions)
print("R-squared score:", r2_dt)


Decision Tree Regression MSE: 413056073.68333334
R-squared score: 0.8432116775940419


In [18]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [19]:
rf_mse = mean_squared_error(y_test, rf_predictions)
print("Random Forest Regression MSE:", rf_mse)

r2_rf = r2_score(y_test, rf_predictions)
print("R-squared score:", r2_rf)

Random Forest Regression MSE: 351219103.79025
R-squared score: 0.8666838291732416


In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Reshape data for LSTM input
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1, X_train.shape[1])))
model.add(Dense(units=1))  # Output layer with 1 neuron for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32)

# Evaluate the model on test data
loss = model.evaluate(X_test_reshaped, y_test)
print("Test loss:", loss)

Epoch 1/50
20/20 [==============================] - 1s 2ms/step - loss: 4167632896.0000
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 4167626240.0000
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 4167616768.0000
Epoch 4/50
20/20 [==============================] - 0s 2ms/step - loss: 4167604224.0000
Epoch 5/50
20/20 [==============================] - 0s 2ms/step - loss: 4167588096.0000
Epoch 6/50
20/20 [==============================] - 0s 2ms/step - loss: 4167565824.0000
Epoch 7/50
20/20 [==============================] - 0s 2ms/step - loss: 4167537920.0000
Epoch 8/50
20/20 [==============================] - 0s 2ms/step - loss: 4167500288.0000
Epoch 9/50
20/20 [==============================] - 0s 2ms/step - loss: 4167453696.0000
Epoch 10/50
20/20 [==============================] - 0s 2ms/step - loss: 4167395328.0000
Epoch 11/50
20/20 [==============================] - 0s 1ms/step - loss: 4167327232.0000
Epoch 12/50
20/20 [===========